# **Preparación de datos previa a proceso de modelado**

## **Librerías**

In [10]:
from prep_auxiliary_functions import (add_sin_cos_transforms, 
                                 )
import warnings
import pandas as pd
import numpy as np

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
warnings.filterwarnings('ignore')

## **Transformaciones**

###
**Lectura y unión de datasets**

In [12]:
energy = pd.read_pickle('energy_clean_dataset.pkl')
weather = pd.read_pickle('weather_clean_dataset.pkl')
df = pd.merge(weather, energy, how="left", on="time")
print("Shape:", df.shape)
df.head(3)

Shape: (178436, 32)


,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_main,weather_description,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual,price day ahead
0,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 00:00:00,447.0,329.0,4844.0,4821.0,298.319789,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0,50.10
1,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 01:00:00,449.0,328.0,5196.0,4755.0,298.319789,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0,48.10
2,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 02:00:00,448.0,323.0,4857.0,4581.0,298.319789,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0,47.33


###
**Transformaciones temporales**

In [13]:
# df = add_sin_cos_transforms(df, ["month", "dayofweek", "hour"])
# df.head(3)

###
**Transformaciones de representación de perídos pasados**

In [ ]:
# la media de la columna X de las ultimas 4 semanas en la hora H como variable predictora de lapso del futuro.
# para hacerlo con clima primero hay que hacer pivot de las ciudades.

## **Modelado predictivo**

El objetivo es predecir las 24 horas de la demanda total de energía con una anticipación de 12,24 y 36 horas según las sesiones intradiarias del mercado eléctrico mayorista.

In [14]:
df

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_main,weather_description,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual,price day ahead
0,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 00:00:00,447.0,329.0,4844.0,4821.0,298.319789,863.000000,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0,50.10
1,Valencia,270.475,270.475,270.475,1001.0,77.0,1.0,62.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 01:00:00,449.0,328.0,5196.0,4755.0,298.319789,920.000000,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0,48.10
2,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 02:00:00,448.0,323.0,4857.0,4581.0,298.319789,1164.000000,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0,47.33
3,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 03:00:00,438.0,254.0,4314.0,4131.0,298.319789,1503.000000,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,22642.0,21286.0,42.27
4,Valencia,269.686,269.686,269.686,1002.0,78.0,0.0,23.0,0.0,0.0,0.0,0.0,clear,sky is clear,2015-01-01 04:00:00,428.0,187.0,4130.0,3840.0,298.319789,475.577343,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,21785.0,20264.0,38.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178431,Seville,287.760,287.150,288.150,1028.0,54.0,3.0,30.0,0.0,0.0,0.0,0.0,clear,sky is clear,2018-12-31 19:00:00,297.0,0.0,7634.0,2628.0,178.000000,1.000000,1135.0,4836.0,6073.0,63.0,95.0,85.0,277.0,3113.0,30619.0,30653.0,68.85
178432,Seville,285.760,285.150,286.150,1029.0,62.0,3.0,30.0,0.0,0.0,0.0,0.0,clear,sky is clear,2018-12-31 20:00:00,296.0,0.0,7241.0,2566.0,174.000000,1.000000,1172.0,3931.0,6074.0,62.0,95.0,33.0,280.0,3288.0,29932.0,29735.0,68.40
178433,Seville,285.150,285.150,285.150,1028.0,58.0,4.0,50.0,0.0,0.0,0.0,0.0,clear,sky is clear,2018-12-31 21:00:00,292.0,0.0,7025.0,2422.0,168.000000,50.000000,1148.0,2831.0,6076.0,61.0,94.0,31.0,286.0,3503.0,27903.0,28071.0,66.88
178434,Seville,284.150,284.150,284.150,1029.0,57.0,4.0,60.0,0.0,0.0,0.0,0.0,clear,sky is clear,2018-12-31 22:00:00,293.0,0.0,6562.0,2293.0,163.000000,108.000000,1128.0,2068.0,6075.0,61.0,93.0,31.0,287.0,3586.0,25450.0,25801.0,63.93


In [15]:
(28191.319789 - 25385.0)/2

1403.1598945000005